# Part 1: FastAPI Testing Notebook

This notebook demonstrates how to test the FastAPI endpoints we've built.

## Prerequisites
1. Start the FastAPI server: `uvicorn app.api.main:app --reload`
2. Server should be running at: http://localhost:8000
3. API docs available at: http://localhost:8000/docs

In [ ]:
import requests
import json
from typing import Dict, Any

# Base URL for our API
BASE_URL = "http://localhost:8000"
API_V1 = f"{BASE_URL}/api/v1"

def pretty_print(response):
    """Helper function to nicely format API responses"""
    print(f"Status: {response.status_code}")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
    print("-" * 50)

## 1. Health Check
Let's start by checking if our API is running properly.

In [ ]:
# Test health endpoint
response = requests.get(f"{BASE_URL}/health")
pretty_print(response)

## 2. Root Endpoint
Check the root endpoint for basic API information.

In [ ]:
# Test root endpoint
response = requests.get(BASE_URL)
pretty_print(response)

## 3. Create Users
Let's create some test users to work with.

In [ ]:
# Test data for creating users
test_users = [
    {
        "name": "Alice Johnson",
        "email": "alice@example.com",
        "age": 28
    },
    {
        "name": "Bob Smith",
        "email": "bob@example.com",
        "age": 35
    },
    {
        "name": "Charlie Brown",
        "email": "charlie@example.com"
        # Note: age is optional
    }
]

created_users = []

# Create each user
for user_data in test_users:
    print(f"Creating user: {user_data['name']}")
    response = requests.post(f"{API_V1}/users", json=user_data)
    pretty_print(response)
    
    if response.status_code == 201:
        created_users.append(response.json())
    
    print()

## 4. List All Users
Test the pagination functionality.

In [ ]:
# Get all users (default pagination)
print("All users (default pagination):")
response = requests.get(f"{API_V1}/users")
pretty_print(response)
print()

# Test pagination parameters
print("First page with size 2:")
response = requests.get(f"{API_V1}/users", params={"page": 1, "size": 2})
pretty_print(response)
print()

print("Second page with size 2:")
response = requests.get(f"{API_V1}/users", params={"page": 2, "size": 2})
pretty_print(response)

## 5. Get Individual Users
Test retrieving specific users by ID.

In [ ]:
# Get first created user if available
if created_users:
    user_id = created_users[0]['id']
    print(f"Getting user with ID {user_id}:")
    response = requests.get(f"{API_V1}/users/{user_id}")
    pretty_print(response)
    print()

# Test non-existent user
print("Trying to get non-existent user (ID 999):")
response = requests.get(f"{API_V1}/users/999")
pretty_print(response)

## 6. Update Users
Test the update functionality.

In [ ]:
if created_users:
    user_id = created_users[0]['id']
    
    # Update user's age
    print(f"Updating user {user_id} age:")
    update_data = {"age": 30}
    response = requests.put(f"{API_V1}/users/{user_id}", json=update_data)
    pretty_print(response)
    print()
    
    # Update user's name and email
    print(f"Updating user {user_id} name and email:")
    update_data = {
        "name": "Alice Cooper",
        "email": "alice.cooper@example.com"
    }
    response = requests.put(f"{API_V1}/users/{user_id}", json=update_data)
    pretty_print(response)
    print()
    
    # Try to update with duplicate email
    if len(created_users) > 1:
        print(f"Trying to update with duplicate email:")
        duplicate_email = created_users[1]['email']
        update_data = {"email": duplicate_email}
        response = requests.put(f"{API_V1}/users/{user_id}", json=update_data)
        pretty_print(response)

## 7. Check User Existence
Test the utility endpoint to check if users exist.

In [ ]:
if created_users:
    user_id = created_users[0]['id']
    print(f"Checking if user {user_id} exists:")
    response = requests.get(f"{API_V1}/users/{user_id}/exists")
    pretty_print(response)
    print()

print("Checking if non-existent user exists:")
response = requests.get(f"{API_V1}/users/999/exists")
pretty_print(response)

## 8. Validation Testing
Test the Pydantic validation by sending invalid data.

In [ ]:
print("Testing validation errors:")
print()

# Invalid email format
print("1. Invalid email format:")
invalid_user = {
    "name": "Invalid User",
    "email": "not-an-email",
    "age": 25
}
response = requests.post(f"{API_V1}/users", json=invalid_user)
pretty_print(response)
print()

# Negative age
print("2. Negative age:")
invalid_user = {
    "name": "Invalid User",
    "email": "invalid@example.com",
    "age": -5
}
response = requests.post(f"{API_V1}/users", json=invalid_user)
pretty_print(response)
print()

# Missing required field
print("3. Missing name field:")
invalid_user = {
    "email": "missing@example.com",
    "age": 25
}
response = requests.post(f"{API_V1}/users", json=invalid_user)
pretty_print(response)
print()

# Empty name
print("4. Empty name:")
invalid_user = {
    "name": "",
    "email": "empty@example.com",
    "age": 25
}
response = requests.post(f"{API_V1}/users", json=invalid_user)
pretty_print(response)

## 9. Delete Users
Test user deletion functionality.

In [ ]:
if created_users:
    # Delete the last created user
    user_to_delete = created_users[-1]
    user_id = user_to_delete['id']
    
    print(f"Deleting user {user_id} ({user_to_delete['name']}):")
    response = requests.delete(f"{API_V1}/users/{user_id}")
    print(f"Status: {response.status_code}")
    print(f"Response: {response.text if response.text else 'No content (expected for 204)'}")
    print("-" * 50)
    print()
    
    # Try to get the deleted user
    print(f"Trying to get deleted user {user_id}:")
    response = requests.get(f"{API_V1}/users/{user_id}")
    pretty_print(response)
    print()
    
    # Try to delete non-existent user
    print("Trying to delete non-existent user (ID 999):")
    response = requests.delete(f"{API_V1}/users/999")
    pretty_print(response)

## 10. Final State Check
Check the final state of users after all operations.

In [ ]:
print("Final user list:")
response = requests.get(f"{API_V1}/users")
pretty_print(response)

print("\n🎉 API testing complete!")
print("\n📝 Key takeaways:")
print("- FastAPI automatically validates input using Pydantic models")
print("- Proper HTTP status codes are returned (201, 204, 400, 404)")
print("- Pagination works correctly with query parameters")
print("- Email validation prevents duplicate entries")
print("- Error messages are clear and helpful")
print("\n🔍 Next: Explore the automatic API docs at http://localhost:8000/docs")

## Bonus: Using the Interactive API Documentation

FastAPI automatically generates interactive API documentation. While the server is running:

1. **Swagger UI**: http://localhost:8000/docs
   - Interactive interface to test all endpoints
   - Shows request/response schemas
   - Try out functionality built-in

1. **OpenAPI Schema**: http://localhost:8000/openapi.json
   - Raw OpenAPI specification
   - Can be used with other tools
   - Machine-readable API definition

These are generated automatically from your code - no extra work needed!